<h1>Load Data</h1>

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

### beers and breweries

In [2]:
beer_file = r'../data/bigData/beers.csv'
breweries_file = r'../data/bigData/breweries.csv'

beers = pd.read_csv (beer_file)

breweries = pd.read_csv (breweries_file)
breweries.rename(columns={'id':'brewery_id'},inplace=True)

In [3]:
reviews = pd.read_csv (r"../data/bigData/reviews.csv")

### long and lat of cities

In [4]:
cities_pos_file  = r'lat_long_03-05-2020_19:21.csv' 

cities_pos = pd.read_csv (cities_pos_file)

### join_beer_brewery

In [5]:
beer_and_breweries = pd.merge(beers,breweries,on="brewery_id")
beer_and_breweries.rename(columns={'name_x':'beer','name_y':'brewery','country_y':'country'},inplace=True)
beer_and_breweries = beer_and_breweries[["id","beer","style","availability","abv","retired","brewery","city","country","types"]]

### cleaning

In [6]:
beer_and_breweries = beer_and_breweries[beer_and_breweries["abv"]<70]


In [7]:
beer_and_breweries["abv"][beer_and_breweries["abv"]==52.0] = 5.2

In [10]:
beer_and_breweries[beer_and_breweries['beer'] == 'Octo-Pyroclastic Black IPA']

,id,beer,style,availability,abv,retired,brewery,city,country,types
290449,94975,Octo-Pyroclastic Black IPA,German Eisbock,Limited (brewed once),24.0,t,La Brasserie de Fleurac,Ydes,FR,Brewery


Delete redondant data

In [7]:
del beers
del breweries

### print data infos

In [8]:
beer_and_breweries.head(2)

,id,beer,style,availability,abv,retired,brewery,city,country,types
0,202522,Olde Cogitator,English Oatmeal Stout,Rotating,7.3,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery"
1,8677,Red Leaf Strong Ale,American Strong Ale,Year-round,8.6,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery"


In [9]:
cities_pos.head(2)

,Country,City,lat,long
0,BE,Erpe-Mere,50.93604,4.03550
1,US,Thousand Palms,33.82002,-116.39029


In [10]:
reviews.head(2)

,beer_id,username,date,text,look,smell,taste,feel,overall,score
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of...",4.0,4.0,4.0,4.25,4.0,4.03
1,125646,_dirty_,2017-12-21,,4.5,4.5,4.5,4.50,4.5,4.50


### add reviews to beer_and_breweries

In [18]:
## clear reviews quickly because too much memory
reviews = reviews[["beer_id",'look', 'smell', 'taste', 'feel',
       'overall', "score"]].groupby("beer_id")

In [ ]:
reviews_avg = reviews.mean()

In [ ]:
reviews_count = reviews.count()

In [21]:
#del reviews

In [22]:
temp = reviews_count*reviews_avg  + 5
temp = temp / (reviews_count+2)
temp.head()

reviews_score_adjusted = temp
del temp

In [27]:
all_d = pd.merge(beer_and_breweries,reviews_score_adjusted,left_on='id',how='left',right_on='beer_id')
temp = pd.merge(beer_and_breweries,reviews_count,left_on='id',how='left',right_on='beer_id')
temp = temp.rename(columns={"score":"popularity"})
temp = pd.merge(all_d,temp[["beer","brewery","country","popularity"]],how='left',on=["beer","brewery","country"])
beer_and_breweries = temp
del temp
del all_d
beer_and_breweries.head()

,id,beer,style,availability,abv,retired,brewery,city,country,types,look,smell,taste,feel,overall,score,popularity
0,202522,Olde Cogitator,English Oatmeal Stout,Rotating,7.3,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8677,Red Leaf Strong Ale,American Strong Ale,Year-round,8.6,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery",3.583333,3.166667,3.416667,3.416667,3.583333,3.400000,4.0
2,202536,Bikini Bottom,American Lager,Rotating,4.9,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202523,Ded Hed Red,American Amber / Red Ale,Rotating,5.6,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8749,Pleasanton Pale Ale,American Pale Ale (APA),Year-round,5.3,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery",3.416667,3.250000,3.416667,3.583333,3.500000,3.387143,5.0


---
# Raw data to decent json

In [57]:
beer_and_breweries.set_index("id").to_json("../data/raw_data.json",orient='records')

In [58]:
bpc = beer_and_breweries.groupby("country").apply(lambda x: x.to_json(orient='records'))
bpc.to_json("../data/raw_data_per_c.json",orient="index")

---
# Infos per country

In [76]:
countries = beer_and_breweries[["country"]].drop_duplicates()
len(countries)


187

In [77]:
get_data_filters(beer_and_breweries,"",countries).head()

,country,n_beers,avg_abv,max_abv_beer,max_abv_brewery,max_abv,best_beer,best_brewery,best_beer_score,popular_beer,popular_brewery,popularity
0,US,234922.0,6.712535,Samuel Adams Utopias,Boston Beer Company (Samuel Adams),29.0,Kentucky Brunch Brand Stout,Toppling Goliath Brewing Company,4.831094,Breakfast Stout,Founders Brewing Company,17160.0
4,NO,1099.0,6.396278,Dark Horizon 4.5,Nøgne Ø - Det Kompromissløse Bryggeri A/S,20.0,Brewers Reserve Barley Wine (Aged In Bourbon B...,Lervig Aktiebryggeri,4.428228,Imperial Stout,Nøgne Ø - Det Kompromissløse Bryggeri A/S,895.0
5,JP,1588.0,5.686366,Hakusekikan Hurricane,"Stone Iwamoto Co., Ltd / Hakusekikan Beer Rest...",15.0,Hitachino Nest White Ale,Kiuchi Brewery,4.067816,Sapporo Premium Beer,Sapporo Breweries Ltd.,2236.0
7,IT,2535.0,6.400773,Strongest Than Ever BA Cognac 2013,White Pony Microbrewery,16.0,Xyauyù Barrel,Birrificio Le Baladin,4.377241,Peroni Nastro Azzurro,Birra Peroni Industriale S.p.A.,2090.0
10,CA,20133.0,6.092153,Glacial Mammoth Extinction,Storm Brewing,25.0,Péché Mortel - Fût De Bourbon Américain,Brasserie Dieu du Ciel!,4.439790,La Fin Du Monde,Unibroue,10295.0


## Beer types

In [ ]:
beer_types = beer_and_breweries[['style','id']].groupby('style').agg('count').reset_index()

In [ ]:
beer_and_breweries = beer_and_breweries.set_index('id')

In [80]:
IPA = beer_and_breweries[beer_and_breweries['style'].str.contains('IPA')]

In [81]:
Lager = beer_and_breweries[(beer_and_breweries['style'].str.contains('Lager'))]

In [82]:
Ale = beer_and_breweries[(beer_and_breweries['style'].str.contains('Ale')) | (beer_and_breweries['style'].str.contains('Gose'))]

In [83]:
Stout = beer_and_breweries[(beer_and_breweries['style'].str.contains('Stout'))]

In [84]:
Pilsner = beer_and_breweries[beer_and_breweries['style'].str.contains('Pils')]

In [85]:
Porter = beer_and_breweries[beer_and_breweries['style'].str.contains('Porter')]

In [86]:
Fruit =beer_and_breweries[beer_and_breweries['style'].str.contains('Fruit')]

In [87]:
Roggenbier = beer_and_breweries[(beer_and_breweries['style'].str.contains('Roggen'))|(beer_and_breweries['style'].str.contains('Rye'))]

In [88]:
Herb = beer_and_breweries[(beer_and_breweries['style'].str.contains('Herb'))]

In [89]:
Smoke = beer_and_breweries[(beer_and_breweries['style'].str.contains('Smoke'))]

In [90]:
Barleywine=beer_and_breweries[(beer_and_breweries['style'].str.contains('Barleywine'))]

In [91]:
def get_data_filters(data,name,countries):
    
    if name !="":
        name = "_"+name
    
    #Number of beers
    temp = data[["country","beer"]].groupby("country").count()
    temp.rename(columns={'beer':'n_beers'+name},inplace=True)
    countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
    
    #Average ABV
    temp = data[["country","abv"]].groupby("country").mean()
    temp.rename(columns={'abv':'avg_abv'+name},inplace=True)
    countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
    
    # HIghest ABV
    #temp = data[["country","abv"]].groupby("country").max()
    #temp.rename(columns={'abv':'max_abv'+name},inplace=True)
    #countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
    
    idx = data[["country","abv"]].groupby(["country"])["abv"].transform(max) == data["abv"]
    temp = data[idx][['country',"beer","brewery","abv"]]
    temp = temp.rename(columns={"beer":"max_abv_beer"+name, "abv":"max_abv"+name,"brewery":"max_abv_brewery"+name})
    countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
    
    # best_beer best_beer_score 
    
    idx = data[["country","score"]].groupby(["country"])["score"].transform(max) == data["score"]
    temp = data[idx][['country',"beer","brewery","score"]]
    temp = temp.rename(columns={"beer":"best_beer"+name, "score":"best_beer_score"+name,"brewery":"best_brewery"+name})
    countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
    
    
    # most_popular_beer popularity
    idx = data[["country","popularity"]].groupby(["country"])["popularity"].transform(max) == data["popularity"]
    temp = data[idx][['country',"beer","brewery","popularity"]]
    temp = temp.rename(columns={"beer":"popular_beer"+name, "popularity":"popularity"+name,"brewery":"popular_brewery"+name})
    countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')

    
    
    return countries.drop_duplicates("country")
    

In [92]:
countries =get_data_filters(beer_and_breweries,"",countries)
countries =get_data_filters(IPA,"IPA",countries)
countries =get_data_filters(Lager,'Lager',countries)
countries =get_data_filters(Ale,'Ale',countries)
countries =get_data_filters(Stout,"Stout",countries)
countries =get_data_filters(Pilsner,'Pilsner',countries)
countries =get_data_filters(Porter,'Porter',countries)
countries =get_data_filters(Fruit,'Fruit',countries)
countries =get_data_filters(Roggenbier,'Roggenbier',countries)
countries =get_data_filters(Herb,'Herb',countries)
countries =get_data_filters(Smoke,'Smoke',countries)
countries =get_data_filters(Barleywine,'Barleywine',countries)

In [93]:
countries.head()

,country,n_beers,avg_abv,max_abv_beer,max_abv_brewery,max_abv,best_beer,best_brewery,best_beer_score,popular_beer,...,avg_abv_Barleywine,max_abv_beer_Barleywine,max_abv_brewery_Barleywine,max_abv_Barleywine,best_beer_Barleywine,best_brewery_Barleywine,best_beer_score_Barleywine,popular_beer_Barleywine,popular_brewery_Barleywine,popularity_Barleywine
0,US,234922.0,6.712535,Samuel Adams Utopias,Boston Beer Company (Samuel Adams),29.0,Kentucky Brunch Brand Stout,Toppling Goliath Brewing Company,4.831094,Breakfast Stout,...,10.831299,The Hedge Hog,Hailstorm Brewing Co.,20.0,King Henry,Goose Island Beer Co.,4.704796,Bigfoot Barleywine-Style Ale,Sierra Nevada Brewing Co.,7627.0
2,NO,1099.0,6.396278,Dark Horizon 4.5,Nøgne Ø - Det Kompromissløse Bryggeri A/S,20.0,Brewers Reserve Barley Wine (Aged In Bourbon B...,Lervig Aktiebryggeri,4.428228,Imperial Stout,...,10.444444,•Ægir Tors Hammar,Ægir Bryggeri,13.2,Brewers Reserve Barley Wine (Aged In Bourbon B...,Lervig Aktiebryggeri,4.428228,# 100,Nøgne Ø - Det Kompromissløse Bryggeri A/S,474.0
3,JP,1588.0,5.686366,Hakusekikan Hurricane,"Stone Iwamoto Co., Ltd / Hakusekikan Beer Rest...",15.0,Hitachino Nest White Ale,Kiuchi Brewery,4.067816,Sapporo Premium Beer,...,10.370588,Hakusekikan Hurricane,"Stone Iwamoto Co., Ltd / Hakusekikan Beer Rest...",15.0,Samurai Barley Ale,Hyouko Yashiki No Mori Brewery,3.772500,Samurai Barley Ale,Hyouko Yashiki No Mori Brewery,46.0
5,IT,2535.0,6.400773,Strongest Than Ever BA Cognac 2013,White Pony Microbrewery,16.0,Xyauyù Barrel,Birrificio Le Baladin,4.377241,Peroni Nastro Azzurro,...,11.893590,Strongest Than Ever BA Cognac 2013,White Pony Microbrewery,16.0,Xyauyù Barrel,Birrificio Le Baladin,4.377241,Xyauyù Barrel,Birrificio Le Baladin,114.0
8,CA,20133.0,6.092153,Glacial Mammoth Extinction,Storm Brewing,25.0,Péché Mortel - Fût De Bourbon Américain,Brasserie Dieu du Ciel!,4.439790,La Fin Du Monde,...,10.204735,Flying Monkeys Riley Wine,Flying Monkeys Craft Brewery,18.0,Solstice D'hiver Réserve Spéciale En Fût De Bo...,Brasserie Dieu du Ciel!,4.147623,Solstice D'hiver,Brasserie Dieu du Ciel!,946.0


## Saving to csv and json

In [94]:
countries=countries.set_index("country")

countries.to_csv(r"../data/countries_basic_data.csv")
countries.to_json(r"../data/countries_basic_data.json",orient='index')

## Reviews for cloud words

In [27]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import string
from tqdm import tqdm_notebook

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
beer_and_breweries

In [28]:
def lower_punct_removal(text):
    temp = text.lower()
    return temp.translate(str.maketrans('', '', string.punctuation))
def stopwords_removal(text):
    obvious= ['beer','nice','like','good','well','bottle','bit','head','taste','flavor','smell','little',\
              'glass','pours','notes','body','carbonation','color','hops','really','mouthfeel','aroma','malt',\
             'one','overall','would','finish','much','medium','poured']
    return ' '.join([x for x in text.split() if not (x in stopwords_en or x.isdigit() or x in obvious) ])



def get_words_counts(beer_sub_type,number):
    barley_reviews = reviews[reviews['beer_id'].isin(beer_sub_type.index.to_list())].copy()
    barley_reviews['text']  = barley_reviews['text'].apply(lower_punct_removal)
    barley_reviews['text']=barley_reviews['text'].apply(stopwords_removal)
    
    counter = CountVectorizer()
    X = counter.fit_transform(list(barley_reviews['text']))
    word_count= np.asarray(X.sum(axis =0))[0]
    
    indices = np.argsort(word_count)[-number:]
    counts = np.sort(word_count)[-number:]
    
    return np.array(counter.get_feature_names())[indices],counts


def store_words_(data,beer_types):
    result = pd.DataFrame(columns=['beer_type','words'])
    result['beer_type'] = beer_types
    for j in tqdm_notebook(range(len(beer_types))):
        words,counts = get_words_counts(data[j],20)
        liste = []
        for i in range(len(words)):
            liste.append({"x":words[i],"value": counts[i]})
        result.loc[j,'words'] = liste

    return result

In [29]:
stopwords_en =stopwords.words('english')

In [30]:
reviews = reviews[reviews['text']!= '\xa0\xa0']

In [31]:
get_words_counts(Barleywine,20)

AttributeError: 'Int64Index' object has no attribute 'to_list'

In [284]:
get_words_counts(IPA,20)

(array(['alcohol', 'sweetness', 'floral', 'hoppy', 'fruit', 'caramel',
        'nose', 'great', 'lacing', 'white', 'bitter', 'grapefruit',
        'sweet', 'pine', 'light', 'bitterness', 'orange', 'citrus', 'hop',
        'ipa'], dtype='<U235'),
 array([115175, 116725, 117310, 118542, 118895, 144788, 147053, 158205,
        195802, 202436, 209962, 213962, 220079, 227207, 230014, 231322,
        239832, 338567, 343219, 355327], dtype=int64))

In [43]:
get_words_counts(Porter,20)

(array(['sweetness', 'coffee', 'flavors', 'smooth', 'bitter', 'malts',
        'great', 'alcohol', 'nose', 'citrus', 'bitterness', 'chocolate',
        'caramel', 'white', 'brown', 'lacing', 'hop', 'dark', 'sweet',
        'light'], dtype='<U235'),
 array([20334, 20734, 22017, 22111, 22662, 22722, 23538, 24320, 24375,
        24634, 25301, 26190, 26721, 27430, 28054, 30211, 31942, 39609,
        45184, 53962], dtype=int64))

In [286]:
get_words_counts(Pilsner,20)

(array(['slightly', 'pale', 'bitter', 'floral', 'dry', 'yellow', 'grassy',
        'style', 'clean', 'lacing', 'bitterness', 'clear', 'pils',
        'golden', 'sweet', 'crisp', 'hop', 'white', 'pilsner', 'light'],
       dtype='<U80'),
 array([14876, 14991, 15144, 15144, 16120, 16292, 16615, 17903, 19306,
        19449, 20514, 21196, 21229, 21730, 23886, 25296, 31371, 32534,
        35235, 51075], dtype=int64))

In [287]:
get_words_counts(Fruit,20)

(array(['flavors', 'great', 'slightly', 'cherries', 'fruity', 'pretty',
        'drink', 'nose', 'sweetness', 'red', 'wheat', 'lacing', 'white',
        'raspberry', 'sour', 'cherry', 'tart', 'fruit', 'light', 'sweet'],
       dtype='<U133'),
 array([13637, 13727, 13817, 14179, 14498, 14540, 14985, 16012, 16625,
        16628, 16672, 17820, 21679, 22410, 24255, 26611, 27783, 37319,
        46716, 48155], dtype=int64))

In [288]:
get_words_counts(Smoke,20)

(array(['smokiness', 'nose', 'tan', 'smooth', 'flavors', 'smokey',
        'lacing', 'smoky', 'malts', 'coffee', 'black', 'sweet', 'roasted',
        'porter', 'brown', 'light', 'chocolate', 'dark', 'smoked', 'smoke'],
       dtype='<U108'),
 array([ 3399,  3444,  3472,  3636,  3835,  4166,  4379,  4461,  4865,
         4938,  4984,  5051,  5552,  6473,  6676,  8264,  8391, 10888,
        11397, 17502], dtype=int64))

In [289]:
get_words_counts(Stout,20)

(array(['great', 'bitter', 'flavors', 'bourbon', 'malts', 'lacing',
        'thick', 'creamy', 'smooth', 'alcohol', 'vanilla', 'brown',
        'light', 'sweet', 'roasted', 'stout', 'black', 'coffee', 'dark',
        'chocolate'], dtype='<U193'),
 array([ 97684,  98334, 103069, 107168, 111018, 111717, 114940, 117970,
        122051, 127354, 132964, 150820, 151876, 168854, 218100, 255488,
        255555, 367834, 379359, 443778], dtype=int64))

In [28]:
beer_types = ['IPA','Lager','Ale','Stout','Pilsner','Porter','Fruit','Roggenbier','Smoke','Barleywine','Herb']
data = [IPA, Lager, Ale, Stout,Pilsner,Porter,Fruit,Roggenbier,Smoke,Barleywine,Herb]

result = store_words_(data,beer_types)

In [29]:
result.set_index('beer_type').to_json("../data/word_cloud.json",orient='index')

In [30]:
result

,beer_type,words
0,IPA,"[{'x': 'alcohol', 'value': 115175}, {'x': 'swe..."
1,Lager,"[{'x': 'sweetness', 'value': 28757}, {'x': 'cl..."
2,Ale,"[{'x': 'sweetness', 'value': 161656}, {'x': 'f..."
3,Stout,"[{'x': 'great', 'value': 97684}, {'x': 'bitter..."
4,Pilsner,"[{'x': 'slightly', 'value': 14876}, {'x': 'pal..."
5,Porter,"[{'x': 'caramel', 'value': 30712}, {'x': 'bitt..."
6,Fruit,"[{'x': 'flavors', 'value': 13637}, {'x': 'grea..."
7,Roggenbier,"[{'x': 'spice', 'value': 3761}, {'x': 'flavors..."
8,Smoke,"[{'x': 'smokiness', 'value': 3399}, {'x': 'nos..."
9,Barleywine,"[{'x': 'malts', 'value': 17560}, {'x': 'sugar'..."


# countries based data

In [95]:
beer_and_breweries.columns

Index(['beer', 'style', 'availability', 'abv', 'retired', 'brewery', 'city',
       'country', 'types', 'look', 'smell', 'taste', 'feel', 'overall',
       'score', 'popularity'],
      dtype='object')

In [43]:
all_d = pd.merge(beer_and_breweries,reviews_score_adjusted,left_on='id',how='left',right_on='beer_id')
all_d.head()

,id,beer,style,availability,abv,retired,brewery,city,country,types,look,smell,taste,feel,overall,score
0,202522,Olde Cogitator,English Oatmeal Stout,Rotating,7.3,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery",NaN,NaN,NaN,NaN,NaN,NaN
1,8677,Red Leaf Strong Ale,American Strong Ale,Year-round,8.6,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery",3.583333,3.166667,3.416667,3.416667,3.583333,3.400000
2,202536,Bikini Bottom,American Lager,Rotating,4.9,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery",NaN,NaN,NaN,NaN,NaN,NaN
3,202523,Ded Hed Red,American Amber / Red Ale,Rotating,5.6,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery",NaN,NaN,NaN,NaN,NaN,NaN
4,8749,Pleasanton Pale Ale,American Pale Ale (APA),Year-round,5.3,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery",3.416667,3.250000,3.416667,3.583333,3.500000,3.387143


In [44]:
all_d = all_d[["beer","style","abv",'brewery','city','country','look', 'smell', 'taste', 'feel',
       'overall', 'score']]

In [45]:
cities_pos.columns

Index(['Country', 'City', 'lat', 'long'], dtype='object')

In [46]:
all_d = pd.merge(all_d,cities_pos,how='left',left_on=['country','city'],right_on=['Country','City'])
all_d = all_d[["beer","style","abv",'brewery','city','country','look', 'smell', 'taste', 'feel',
       'overall', 'score','lat','long']]
all_d.head()

,beer,style,abv,brewery,city,country,look,smell,taste,feel,overall,score,lat,long
0,Olde Cogitator,English Oatmeal Stout,7.3,Main Street Brewery,Pleasanton,US,NaN,NaN,NaN,NaN,NaN,NaN,38.17781,-94.71135
1,Red Leaf Strong Ale,American Strong Ale,8.6,Main Street Brewery,Pleasanton,US,3.583333,3.166667,3.416667,3.416667,3.583333,3.400000,38.17781,-94.71135
2,Bikini Bottom,American Lager,4.9,Main Street Brewery,Pleasanton,US,NaN,NaN,NaN,NaN,NaN,NaN,38.17781,-94.71135
3,Ded Hed Red,American Amber / Red Ale,5.6,Main Street Brewery,Pleasanton,US,NaN,NaN,NaN,NaN,NaN,NaN,38.17781,-94.71135
4,Pleasanton Pale Ale,American Pale Ale (APA),5.3,Main Street Brewery,Pleasanton,US,3.416667,3.250000,3.416667,3.583333,3.500000,3.387143,38.17781,-94.71135


In [71]:
temp = pd.merge(beer_and_breweries,reviews_count,left_on='id',how='left',right_on='beer_id')
temp = temp.rename(columns={"score":"popularity"})
temp = pd.merge(all_d,temp[["beer","brewery","country","popularity"]],how='left',on=["beer","brewery","country"])
all_d = temp
del temp

In [72]:
breweries_d = all_d.groupby(["brewery",'country',"city","lat","long"])

In [51]:
brewery_to_save = all_d[["brewery",'country',"city","lat","long"]].drop_duplicates()
brewery_to_save.head()


,brewery,country,city,lat,long
0,Main Street Brewery,US,Pleasanton,38.17781,-94.71135
14,Lervig Aktiebryggeri,NO,Stavanger,58.97005,5.73332
105,Byway Brewing,US,Hammond,44.97886,-92.43575
129,Georgetown Brewing Company,US,Seattle,47.60621,-122.33207
290,Mad Princes Brewing,US,Doylestown,40.97005,-81.69652


#### Add aggregated data

1) best beer per brewery

In [55]:
idx = breweries_d["score"].transform(max) == all_d["score"]
best_beer = all_d[idx][["brewery",'country',"city","lat","long","beer","score"]]
best_beer.head()

,brewery,country,city,lat,long,beer,score
1,Main Street Brewery,US,Pleasanton,38.177810,-94.711350,Red Leaf Strong Ale,3.400000
41,Lervig Aktiebryggeri,NO,Stavanger,58.970050,5.733320,Brewers Reserve Barley Wine (Aged In Bourbon B...,4.428228
125,Byway Brewing,US,Hammond,44.978860,-92.435750,Oat Street,3.562000
150,Georgetown Brewing Company,US,Seattle,47.606210,-122.332070,Lucille IPA,4.152405
297,Mad Princes Brewing,US,Doylestown,40.970050,-81.696520,Conniption,3.320000
309,Ezo Beer KK,JP,Hokkaido Sapporo-shi,42.975000,144.374720,Chocolate Bear Stout (Rogue's Chocolate Stout),3.848000
345,Cismontane Brewing Company,US,Rancho Santa Margarita,33.640860,-117.603100,Black's Nocturne,4.153918
436,Hog Haus Brewing Company,US,Fayetteville,38.052890,-81.103990,IPA,3.498462
456,Paddock Wood Brewing Co.,CA,Saskatoon,52.132380,-106.668920,Barrel Full Of Monkeys Imperial Stout,4.152558
510,Woolshed Brewery,AU,Murtho,-34.074476,140.852623,Firehouse Coffee Stout,3.192500


In [59]:
temp = pd.merge(brewery_to_save,best_beer,on=["brewery",'country',"city","lat","long"],how="left")
temp = temp.rename(columns={"beer": "best_beer", "score": "best_beer_score"})
temp.head()

,brewery,country,city,lat,long,best_beer,best_beer_score
0,Main Street Brewery,US,Pleasanton,38.17781,-94.71135,Red Leaf Strong Ale,3.400000
1,Lervig Aktiebryggeri,NO,Stavanger,58.97005,5.73332,Brewers Reserve Barley Wine (Aged In Bourbon B...,4.428228
2,Byway Brewing,US,Hammond,44.97886,-92.43575,Oat Street,3.562000
3,Georgetown Brewing Company,US,Seattle,47.60621,-122.33207,Lucille IPA,4.152405
4,Mad Princes Brewing,US,Doylestown,40.97005,-81.69652,Conniption,3.320000


In [60]:
brewery_to_save = temp

2) most popular beer per brewery

In [73]:
idx = breweries_d["popularity"].transform(max) == all_d["popularity"]
best_beer = all_d[idx][["brewery",'country',"city","lat","long","beer","popularity"]]
best_beer.head()

,brewery,country,city,lat,long,beer,popularity
4,Main Street Brewery,US,Pleasanton,38.17781,-94.71135,Pleasanton Pale Ale,5.0
14,Lervig Aktiebryggeri,NO,Stavanger,58.97005,5.73332,Konrads Stout Russian Imperial Stout,99.0
120,Byway Brewing,US,Hammond,44.97886,-92.43575,ChiPa,11.0
189,Georgetown Brewing Company,US,Seattle,47.60621,-122.33207,Manny's Pale Ale,404.0
297,Mad Princes Brewing,US,Doylestown,40.97005,-81.69652,Conniption,3.0


In [75]:
temp = pd.merge(brewery_to_save,best_beer,on=["brewery",'country',"city","lat","long"],how="left")
temp = temp.rename(columns={"beer": "most_popular_beer"})
brewery_to_save = temp

3) number of beers

In [78]:
n_beers = breweries_d[["beer"]].count()
n_beers.head()

,,,,,beer
brewery,country,city,lat,long,
#Freedom Brewery,US,Buchanan,33.80260,-85.18856,3
'A Magara,IT,Nocera Terinese,39.03474,16.16737,6
't Brugs Bierinstituur,BE,Brugge,51.20892,3.22424,5
't Hofbrouwerijke,BE,Beerzel,51.05753,4.67127,30
(512) Brewing Company,US,Austin,41.63118,-78.09139,60


temp = pd.merge(brewery_to_save,n_beers,on=["brewery",'country',"city","lat","long"],how="left")
temp = temp.rename(columns={"beer": "n_beers"})
temp.head()

In [80]:
brewery_to_save = temp

4) Mean ABV

In [81]:
mean_abv = breweries_d[["abv"]].mean()
mean_abv.head()

,,,,,abv
brewery,country,city,lat,long,
#Freedom Brewery,US,Buchanan,33.80260,-85.18856,5.366667
'A Magara,IT,Nocera Terinese,39.03474,16.16737,6.266667
't Brugs Bierinstituur,BE,Brugge,51.20892,3.22424,7.000000
't Hofbrouwerijke,BE,Beerzel,51.05753,4.67127,7.886667
(512) Brewing Company,US,Austin,41.63118,-78.09139,7.080000


In [83]:
temp = pd.merge(brewery_to_save,mean_abv,on=["brewery",'country',"city","lat","long"],how="left")
temp = temp.rename(columns={"abv": "avg_abv"})
temp.head()

,brewery,country,city,lat,long,best_beer,best_beer_score,most_popular_beer,popularity,n_beers,avg_abv
0,Main Street Brewery,US,Pleasanton,38.17781,-94.71135,Red Leaf Strong Ale,3.400000,Pleasanton Pale Ale,5.0,14.0,6.221429
1,Lervig Aktiebryggeri,NO,Stavanger,58.97005,5.73332,Brewers Reserve Barley Wine (Aged In Bourbon B...,4.428228,Konrads Stout Russian Imperial Stout,99.0,91.0,7.431868
2,Byway Brewing,US,Hammond,44.97886,-92.43575,Oat Street,3.562000,ChiPa,11.0,24.0,6.402083
3,Georgetown Brewing Company,US,Seattle,47.60621,-122.33207,Lucille IPA,4.152405,Manny's Pale Ale,404.0,161.0,6.543727
4,Mad Princes Brewing,US,Doylestown,40.97005,-81.69652,Conniption,3.320000,Conniption,3.0,14.0,5.700000


In [84]:
brewery_to_save = temp

5) Max ABV

In [85]:
max_abv = breweries_d[["abv"]].max()
max_abv.head()

,,,,,abv
brewery,country,city,lat,long,
#Freedom Brewery,US,Buchanan,33.80260,-85.18856,6.5
'A Magara,IT,Nocera Terinese,39.03474,16.16737,8.0
't Brugs Bierinstituur,BE,Brugge,51.20892,3.22424,9.0
't Hofbrouwerijke,BE,Beerzel,51.05753,4.67127,12.5
(512) Brewing Company,US,Austin,41.63118,-78.09139,10.4


In [86]:
temp = pd.merge(brewery_to_save,max_abv,on=["brewery",'country',"city","lat","long"],how="left")
temp = temp.rename(columns={"abv": "max_abv"})
temp.head()

,brewery,country,city,lat,long,best_beer,best_beer_score,most_popular_beer,popularity,n_beers,avg_abv,max_abv
0,Main Street Brewery,US,Pleasanton,38.17781,-94.71135,Red Leaf Strong Ale,3.400000,Pleasanton Pale Ale,5.0,14.0,6.221429,8.6
1,Lervig Aktiebryggeri,NO,Stavanger,58.97005,5.73332,Brewers Reserve Barley Wine (Aged In Bourbon B...,4.428228,Konrads Stout Russian Imperial Stout,99.0,91.0,7.431868,14.0
2,Byway Brewing,US,Hammond,44.97886,-92.43575,Oat Street,3.562000,ChiPa,11.0,24.0,6.402083,10.0
3,Georgetown Brewing Company,US,Seattle,47.60621,-122.33207,Lucille IPA,4.152405,Manny's Pale Ale,404.0,161.0,6.543727,11.4
4,Mad Princes Brewing,US,Doylestown,40.97005,-81.69652,Conniption,3.320000,Conniption,3.0,14.0,5.700000,9.2


In [87]:
brewery_to_save = temp

### Save to independent files

In [118]:
countries = beer_and_breweries["country"].unique()
countries =countries[~pd.isnull(countries)]

brewery_to_save = brewery_to_save.drop_duplicates(subset=["brewery",'country',"city","lat","long"])

for country in countries:
    data = brewery_to_save[brewery_to_save["country"] == country]
    data = data.drop(columns="country")
    data.to_json("../data/country_data/"+country+".json",orient="records")
    

In [121]:
data = brewery_to_save[brewery_to_save["country"] == "CH"]
data = data.drop(columns="country")
data

,brewery,city,lat,long,best_beer,best_beer_score,most_popular_beer,popularity,n_beers,avg_abv,max_abv
647,BFM (Brasserie des Franches-Montagnes),Saignelégier,47.256190,6.996480,Abbaye De Saint Bon-Chien,4.320026,Abbaye De Saint Bon-Chien,1140.0,80.0,8.825375,19.50
996,Brauerei H. Müller AG,Baden,47.473330,8.305920,Dry Stout,2.952500,Muller Brau,6.0,4.0,5.250000,6.20
2388,Öufi Brauerei,Solothurn,47.207910,7.537140,Öufi-Bier Helles Lager,3.206000,Öufi-Bier Helles Lager,3.0,2.0,4.850000,5.00
2877,Brauerei Schützengarten AG,St. Gallen,47.243680,8.964380,St.Galler Landbier,3.731667,St.Galler Klosterbräu,12.0,8.0,5.162500,5.60
3153,Patschiifig Bräu,Brugg,47.480960,8.208690,NaN,NaN,NaN,NaN,2.0,4.800000,4.80
3476,Bier Factory Rapperswil AG,Rapperswil-Jona,47.229828,8.859651,Big Blackbier,3.556000,XXA IPA,6.0,19.0,6.171053,9.60
3918,4655 Brewing Company,Stüsslingen,47.392886,7.971289,NaN,NaN,NaN,NaN,9.0,5.438889,6.22
3997,Culina-Bräu,Höri,47.507990,8.512030,NaN,NaN,NaN,NaN,8.0,4.912500,5.20
4501,Unser Bier,Basel,47.558390,7.573270,Unser Bier Weizen,3.336667,Unser Bier Amber,6.0,6.0,5.183333,6.30
4673,Feldschlösschen Beverages Group,Rheinfelden,47.554370,7.794030,Maibock,3.345714,Feldschlossen Original Lager,90.0,29.0,5.265517,7.20
